# Transaction Snapshot

The main component of snapshot isolation providing the actual state of the snapshot for a given query.

This is using the `CLOG` to create a snapshot for any query / transaction (depending on isolation level).

## Functionality

Each transaction works with its own data `snapshot` / `snapshots`, which makes sure the transaction sees the appropriate data based on `xid`

<img src="./helpers/snapshot-example.png" alt="drawing" width=800 style="background-color:white"/>

## Isolation Levels

### Read Committed

A new snapshot is created on every statement in the transaction

<img src="./helpers/snapshot-read-committed.png" alt="drawing" width=800 style="background-color:white"/>

### Repeatable Read / Serializable

A single snapshot is created at the beginning of the transaction

<img src="./helpers/snapshots-repeatable-read.png" alt="drawing" width=800 style="background-color:white"/>

## Structure

`xmin`:`xmax`:`xip_list`

- xmin - (earliest xid that is still active): All earlier transactions will either be committed and visible, or rolled back and dead.
- xmax - (first as-yet-unassigned txid): All txids greater than or equal to this are not yet started as of the time of the snapshot, and thus invisible.
- xip_list (list of active transaction ids at the time of the snapshot): The list includes only active txids between xmin and xmax.

So the visible transactions are:
- Tn < xmin
- xmin < Tn < xmax `AND` NOT IN xip_list
- Tn = `MYSELF`

That should give you the intuition to understand how the snapshot will be represented.

## Hands On

### Outside Of Transaction

In [ ]:
psql -c "SELECT pg_current_snapshot(), pg_current_xact_id_if_assigned();"

### Inside A Transaction

In [ ]:
# Current snapshot inside a transaction
psql << EOM
BEGIN;
    CREATE TEMP TABLE tbl_a(data int);
    SELECT pg_current_snapshot(), pg_current_xact_id_if_assigned() INTO TEMP TABLE output;
END;
SELECT * FROM output;
EOM

### Different Isolation Levels

This exercise should be done on 4 concurrent sessions (psql / pgadmin doesn't matter).

**Note** - Remember, the default isolation level is `READ COMMITTED`.

In [ ]:
# Prepare env
psql -c "DROP TABLE IF EXISTS a; CREATE TABLE a(data int);"

In [ ]:
-- First Session
BEGIN;
INSERT INTO a VALUES (1);
SELECT pg_current_xact_id_if_assigned();
SELECT pg_current_snapshot();

-- Second Session
BEGIN;
INSERT INTO a VALUES (1);
SELECT pg_current_xact_id_if_assigned();
SELECT pg_current_snapshot();

-- Third Session
BEGIN ISOLATION LEVEL REPEATABLE READ;
INSERT INTO a VALUES (1);
SELECT pg_current_xact_id_if_assigned();
SELECT pg_current_snapshot();

-- Fourth Session
BEGIN;
INSERT INTO a VALUES (1);
SELECT pg_current_xact_id_if_assigned();
SELECT pg_current_snapshot();

-- Fifth Session
BEGIN;
INSERT INTO a VALUES (1);
SELECT pg_current_xact_id_if_assigned();
SELECT pg_current_snapshot();

-- Fourth Session
COMMIT;

-- Run First, Second, Third, Fifth Sessions
SELECT pg_current_snapshot();